# Preparación datos - Tablero Construcción

## Importación de librerías

In [0]:
import pandas as pd
import pyproj
import geopandas as gpd
import matplotlib

import geopandas as gpd
from shapely import wkt


from shapely.geometry import Point
from shapely.wkt import loads

import unicodedata
import requests
import json
import os
import re
import numpy as np

from fuzzywuzzy import fuzz

import jenkspy

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col, cast, when, udf, struct, from_json, count, countDistinct, lit
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType
from pyspark import StorageLevel

from sedona.spark import *
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.register.geo_registrator import SedonaRegistrator
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.utils.adapter import Adapter
from sedona.core.enums import IndexType
from sedona.core.enums import GridType
from sedona.core.spatialOperator import JoinQuery

from sedona.core.SpatialRDD import PointRDD



## Parametrización

In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
num_partitions = 4
url = "https://catalogopmb.catastrobogota.gov.co/PMBWeb/web/api"
startpath = '/dbfs/FileStore/tables/Catastro/'

In [0]:
# configuración de spark y Sedona para datos geograficos
spark = SparkSession.\
    builder.\
    master("local[*]").\
    appName("Sedona App").\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName) .\
    config("spark.kryoserializer.buffer.max", "200gb").\
    getOrCreate()

SedonaContext.create(spark)


sc = spark.sparkContext
sc.setSystemProperty("sedona.global.charset", "utf8")

In [0]:
usos_licencias_homologados = {
    'Residencial':['Vivienda'],
    'Comercio y servicios':['Comercio','Oficinas','Servicios'],
    'Industrial':['Industria'],
    'Otros':['Dotacionales','Institucional','Estacionamientos / Parqueaderos','Otros','Sin informacion','Sin Información','Recreativos']
}

In [0]:
modalidades_homologadas = {
    'Obra nueva':['1.Obra Nueva','1.Obra nueva'],
    'Ampliación':['2.Ampliación'],
    'Modificación':['Modificación'],
    'Demolición':['Demolición Total', 'Demolición Parcial', 'Demolición'],
    'Reforzamiento Estructural':['Reforzamiento Estructural'],
    'Restauración':['Restauración'],
    'Cerramiento':['Cerramiento'],
    'Adecuación':['Adecuación'],
    'Subdivisión':['Subdivisión urbana', 'Subdivisión rural'],
    'Propiedad Horizontal':['Propiedad Horizontal'],
    'Sin Modalidad':['Sin Modalidad', 'Ninguna', 'Sin Información', 'No Aplica Modalidad'],
    'Culminación de Obras':['Culminación de Obras'],
    'Reconocimiento':['Reconocimiento'],
    'Desarrollo':['Desarrollo','Desarrollo por etapas'],
    'Otras':['Restitución','Reloteo', 'Área Bruta Urbanismo', 'Área Útil Urbanismo', 'Saneamiento', 'Reconstrucción'],
}

## Funciones

In [0]:
def remove_words_from_string(input_str, words_to_remove):
    for word in words_to_remove:
        input_str = input_str.replace(word, "")
    return input_str

In [0]:
def clean_ph_address(s):
    s = re.sub(r'\b(AP|GJ|IN|LT) \d+\b', '', s).strip()
    if 'SUR' in s:
        s = re.sub(r'\bSUR\b', '', s).strip() + ' S'
    return s

In [0]:
schema = StructType([StructField("estado", StringType()), StructField("yinput", DoubleType()), StructField("lotcodigo", StringType()),
                    StructField("latitude", StringType()), StructField("diraprox", StringType()), StructField("mancodigo", StringType()),
                    StructField("cpocodigo", StringType()), StructField("xinput", DoubleType()), StructField("codloc", StringType()),
                    StructField("dirtrad", StringType()), StructField("nomupz", StringType()), StructField("localidad", StringType()),
                    StructField("dirinput", StringType()), StructField("codupz", StringType()), StructField("nomseccat", StringType()),
                    StructField("tipo_direccion", StringType()), StructField("codseccat", StringType()), StructField("longitude", StringType()),
                ])
def georeferenciar(input_str:str):
    """
    Realiza la georreferenciación de una cadena de entrada utilizando una API de geocodificación.

    Args:
        input_str (str): La cadena a georreferenciar.

    Returns:
        dict: Un diccionario que contiene los datos de georreferenciación si la operación fue exitosa.
              En caso contrario, devuelve un diccionario que indica un estado de falla o un estado de error.

    """
    schema = StructType([StructField("estado", StringType()), StructField("yinput", DoubleType()), StructField("lotcodigo", StringType()),
                    StructField("latitude", StringType()), StructField("diraprox", StringType()), StructField("mancodigo", StringType()),
                    StructField("cpocodigo", StringType()), StructField("xinput", DoubleType()), StructField("codloc", StringType()),
                    StructField("dirtrad", StringType()), StructField("nomupz", StringType()), StructField("localidad", StringType()),
                    StructField("dirinput", StringType()), StructField("codupz", StringType()), StructField("nomseccat", StringType()),
                    StructField("tipo_direccion", StringType()), StructField("codseccat", StringType()), StructField("longitude", StringType()),
                ])

    input_str = '' if input_str is None else input_str

    direc = unicodedata.normalize("NFKD", input_str).encode("ascii","ignore").decode("ascii").lower()

    val = [True for t in direc.replace('.',' ').replace("  "," ").split() if t in ['via','km','chia','cota','tocancipa','tenjo','madrid','tabio,','cajica,','mosquera','zipaquira']]
    
    if sum(val)>=1:
        result = {'estado': 'outside_bogota','yinput':0,'lotcodigo':'','latitude':'','diraprox':'','mancodigo':'','cpocodigo':'','xinput':0,'codloc':'','dirtrad':'','nomupz':'','localidad':'','dirinput':'','codupz':'','nomseccat':'','tipo_direccion':'','codseccat':'','longitude':''}
        return result

    else:
        try:
            words_to_remove = ["costado occ.", "esquina"]
            direc = remove_words_from_string(direc, words_to_remove)
            direc = clean_ph_address(direc)

            pattern = r'av\.?(?: de la)? esp\.?|\bavenida la esperanza\b|\besperanza\b'
            replacement_str = "AC 24"
            output_str = re.sub(pattern, replacement_str, direc)

            pattern = r'av\.?(?: de las)? ame\.?|\bavenida de las americas\b|\bamericas\b'
            replacement_str = "AC 9"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'(autop\.?(\s)?norte|autonorte|au\.?(\s)?norte|aut\.?(\s)?norte)'
            replacement_str = "AK 45"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'av\.?(?: de las)?(?: c\.?)?\s*(?:ciudad\s*)?cali|avenida\s*cali'
            replacement_str = "AK 86"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'av. nqs|\bnqs\b'
            replacement_str = "AK 30"
            output_str = re.sub(pattern, replacement_str, output_str)

            pattern = r'av. suba|\bsuba\b|tr. suba'
            replacement_str = "AK 69"
            output_str = re.sub(pattern, replacement_str, output_str)

            output_str = output_str.replace('cl. av.', 'AC')
            output_str = output_str.replace('av. cl.', 'AC')
            output_str = output_str.replace('av. cr.', 'AK')

            con = False

            output_str = output_str.replace('con','#')
            output_str = output_str.replace('paralela','#')

            parts = output_str.split("#")

            if len(parts) > 1:
                if '-' not in parts[1]:
                    parts[1] = re.sub(r'[a-zA-Z.]', '', parts[1])
                    output_str = "#".join(parts)

            

            pattern = r'(\d+)\D+(\w+)'
            output_str = re.sub(pattern, lambda x: x.group(1) + ' # ' + x.group(2), output_str)

            if '-' not in output_str:
                output_str = output_str + ' - 01'


            parts = output_str.split("#")

            if parts[0].strip() == 'AK 30' and (int(parts[1].split('-')[0].strip())>100):
                parts[0] = 'AK 9'
                output_str = "#".join(parts)

            output_str = output_str.replace("AK 69",'Av Suba')

            print(output_str)

            params = {
                'cmd': 'geocodificar',
                'apikey': '1c025f92-4520-49c1-90a7-a24b3d9d374c',
                'query': output_str
            }

            response = requests.get(url, params=params)
            res = response.json()
            success = res['response']['success']
            if success:
                result = res['response']['data']
                return result
            else:
                result = {'estado': 'failed','yinput':0,'lotcodigo':'','latitude':'','diraprox':'','mancodigo':'','cpocodigo':'','xinput':0,'codloc':'','dirtrad':'','nomupz':'','localidad':'','dirinput':'','codupz':'','nomseccat':'','tipo_direccion':'','codseccat':'','longitude':''}
                return result
        except:
            result = {'estado': 'error_ws','yinput':0,'lotcodigo':'','latitude':'','diraprox':'','mancodigo':'','cpocodigo':'','xinput':0,'codloc':'','dirtrad':'','nomupz':'','localidad':'','dirinput':'','codupz':'','nomseccat':'','tipo_direccion':'','codseccat':'','longitude':''}
            
            return result
        
georeferenciar_udf = udf(georeferenciar, returnType=schema)

In [0]:
create_point_udf = udf(lambda x,y: Point(x,y).wkt, returnType=StringType())

In [0]:
def homologar_uso_licencia(uso:str,usos_agregados=usos_licencias_homologados):
    '''
    Toma el uso de la base de predios de catastro y lo agrega a 
    categorias mas sencillas de entender para el análisis de datos del OOVS

    Args:
        input_str (str): Cadena de texto que trae el uso para gregarlo.

    Returns:
        str: retorna el uso agragado y homologado por el equipo del observatorio
    '''
    for k,v in usos_agregados.items():
        if uso in v:
            return k
        elif uso not in v:
            pass
homoUsoLicencia_udf = udf(homologar_uso_licencia, StringType())

In [0]:
def homologar_modalidad(modalidad_licencia:str,modalidades_agregados=modalidades_homologadas):
    '''
    Toma el uso de la base de predios de catastro y lo agrega a 
    categorias mas sencillas de entender para el análisis de datos del OOVS

    Args:
        input_str (str): Cadena de texto que trae el uso para gregarlo.

    Returns:
        str: retorna el uso agragado y homologado por el equipo del observatorio
    '''
    for k,v in modalidades_agregados.items():
        if modalidad_licencia in v:
            return k
        elif modalidad_licencia not in v:
            pass
homoModalidad_udf = udf(homologar_modalidad, StringType())

In [0]:
def transform_coordinates(x, y):
    if x is not None and y is not None and x != 'NULL' and y!='NULL' and x != 0.0 and y!=0.0 and isinstance(x, (float)) and isinstance(y, (float)):
        source_crs = pyproj.CRS("ESRI:102233")
        target_crs = pyproj.CRS("EPSG:4326")
        transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
        new_x, new_y = transformer.transform(x, y)
        return new_x, new_y
    else:
        return None, None

coordinate_udf = udf(transform_coordinates, StructType([StructField("wgs84_lng", DoubleType(), False),
                                                        StructField("wgs84_lat", DoubleType(), False)]))

In [0]:
def get_year_from_path(path, paths_dict):
    # Iterate over the dictionary to find the year for the given path
    for year, paths in paths_dict.items():
        if path in paths:
            return year
    return None

## Cargue de datos

### Licencias (SDP)

In [0]:
# Definimos la ubicación del archivo que queremos leer.
file_location = "/FileStore/tables/SDP/sdp_licencias_2008_2023.csv"

# Especificamos el tipo de archivo. En este caso, es un archivo CSV.
file_type = "csv"

# Definimos algunas opciones para la lectura del archivo:
# - 'infer_schema': Si es "true", Spark intentará inferir automáticamente el esquema (tipos de datos) de las columnas.
# - 'first_row_is_header': Si es "true", Spark tratará la primera fila del archivo CSV como encabezados (nombres de columnas).
# - 'delimiter': Especifica el delimitador utilizado en el archivo CSV. En este caso, es una coma.
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Usamos el método 'read' de Spark para leer el archivo CSV con las opciones especificadas.
# El resultado es un DataFrame llamado 'df_licencias' que contiene los datos del archivo CSV.
df_licencias = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)


#### Georeferenciación de licencias SDP a partir de dirección catastral

In [0]:
# df_licencias = df_licencias.withColumn('geocode_result', struct(georeferenciar_udf(col('Dirección')).alias('geocoding')))

In [0]:
# df_licencias = df_licencias.withColumn('estado', col('geocode_result.geocoding.estado'))
# df_licencias = df_licencias.withColumn('tipo_direccion', col('geocode_result.geocoding.tipo_direccion'))
# df_licencias = df_licencias.withColumn('diraprox', col('geocode_result.geocoding.diraprox'))
# df_licencias = df_licencias.withColumn('lat', col('geocode_result.geocoding.yinput'))
# df_licencias = df_licencias.withColumn('lng', col('geocode_result.geocoding.xinput'))
# df_licencias = df_licencias.withColumn('lotcodigo', col('geocode_result.geocoding.lotcodigo'))
# df_licencias = df_licencias.withColumn('eq_lot_georef', col('lotcodigo').cast(LongType()) == col('Código lote').cast(LongType()))

In [0]:
# df_licencias = df_licencias.drop('geocode_result')

In [0]:
# df_licencias.write.csv('/FileStore/tables/SDP/sdp_licencias_2008_2023_georef.csv', header=True, mode="overwrite")

In [0]:
df_licencias = spark.read.csv('/FileStore/tables/SDP/sdp_licencias_2008_2023_georef.csv', header=True)

In [0]:
# Agregamos una nueva columna "new_coords" al DataFrame df_licencias.
# Esta columna se genera utilizando la función definida por el usuario 'coordinate_udf',
# que toma como entrada las columnas "Longitud" y "Latitud".
df_licencias = df_licencias.withColumn("new_coords", coordinate_udf("Longitud", "Latitud"))

# Seleccionamos todas las columnas del DataFrame original, y adicionalmente extraemos
# los campos 'wgs84_lng' y 'wgs84_lat' de la columna "new_coords".
# Posteriormente, eliminamos la columna "new_coords" para mantener solo los campos extraídos.
df_licencias = df_licencias.select("*", "new_coords.wgs84_lng", "new_coords.wgs84_lat").drop("new_coords")


In [0]:
# Actualizamos el DataFrame df_licencias agregando una nueva columna "def_lng".
# Si "wgs84_lng" no es nulo y es diferente de 0, se usa el valor de "wgs84_lng".
# En caso contrario, se utiliza el valor de la columna "lng".
df_licencias = df_licencias.withColumn(
    "def_lng",
    when(
        (col("wgs84_lng").isNotNull()) & 
        (col("wgs84_lng") != 0), 
        col("wgs84_lng")
    ).otherwise(col("lng"))
)

# Similar al proceso anterior, actualizamos el DataFrame df_licencias agregando una nueva columna "def_lat".
# Si "wgs84_lat" no es nulo y es diferente de 0, se usa el valor de "wgs84_lat".
# En caso contrario, se utiliza el valor de la columna "lat".
df_licencias = df_licencias.withColumn(
    "def_lat",
    when(
        (col("wgs84_lat").isNotNull()) & 
        (col("wgs84_lat") != 0), 
        col("wgs84_lat")
    ).otherwise(col("lat"))
)

# Definimos una lista de columnas que queremos eliminar del DataFrame.
columns_to_drop = ["Latitud", "Longitud", "wgs84_lng", "wgs84_lat", "lat", "lng", "eq_lot_georef"]

# Eliminamos las columnas especificadas en la lista 'columns_to_drop' del DataFrame df_licencias.
df_licencias = df_licencias.drop(*columns_to_drop)


In [0]:
# Actualizamos el DataFrame df_licencias agregando una nueva columna "ModalidadAgregada".
# Esta columna se genera utilizando la función definida por el usuario 'homoModalidad_udf',
# que toma como entrada la columna "Modalidad" del DataFrame.
df_licencias = df_licencias.withColumn("ModalidadAgregada", homoModalidad_udf(df_licencias["Modalidad"]))

# Eliminamos la columna "Modalidad" del DataFrame df_licencias ya que ha sido reemplazada por "ModalidadAgregada".
df_licencias = df_licencias.drop("Modalidad")


In [0]:
# Actualizamos el DataFrame df_licencias agregando una nueva columna "UsoAgregado".
# Esta columna se genera utilizando la función definida por el usuario 'homoUsoLicencia_udf',
# que toma como entrada la columna "Uso" del DataFrame.
df_licencias = df_licencias.withColumn("UsoAgregado", homoUsoLicencia_udf(df_licencias["Uso"]))

# Eliminamos la columna "Modalidad" del DataFrame df_licencias ya que ha sido reemplazada por "ModalidadAgregada".
df_licencias = df_licencias.drop("Uso")


In [0]:
# Definimos una lista de tipos de trámites que queremos filtrar en el DataFrame.
tipo_tramite = ['Licencia de Construcción','Reconocimiento de la existencia de una construcción','Licencia de Urbanización','Licencia de Urbanismo y construcción','Propiedad Horizontal','Licencia de Subdivisión','Revalidación licencia de construcción','Autorización para el movimiento de tierras','Sin informacion','Modificación','Ajuste de cotas de áreas','Copia Certificada de planos','Sin Tipo de Trámite','Inicial','Modificación de planos urbanísticos']
tipo_tramite = ['Licencia de Construcción']

# Filtramos el DataFrame df_licencias para mantener solo las filas donde 'Tipo Trámite' esté en la lista tipo_tramite.
df_licencias = df_licencias.filter(col('Tipo Trámite').isin(tipo_tramite))

objeto_tramite = ['Inicial','Modificación']

df_licencias = df_licencias.filter(col('Objeto trámite').isin(objeto_tramite))

# Definimos una lista de modalidades que queremos filtrar en el DataFrame.
modalidades = ['Obra nueva','Ampliación','Modificación','Demolición','Reforzamiento Estructural','Restauración','Cerramiento','Adecuación','Subdivisión','Propiedad Horizontal','Sin Modalidad','Culminación de Obras','Reconocimiento','Desararollo','Otras']
modalidades = ['Obra nueva']

# Filtramos el DataFrame df_licencias para mantener solo las filas donde 'ModalidadAgregada' esté en la lista modalidades.
df_licencias = df_licencias.filter(col('ModalidadAgregada').isin(modalidades))

# Definimos una lista de decisiones que queremos filtrar en el DataFrame.
#decidiones = ['Aprobado', 'Desistido','Aclarado','Prorrogado','Negado','Recurso Confirmado', 'Recurso Rechazado', 'Recurso Aclarado', 'Renuncia de Licencia', 'Revocado', 'Archivado']
#decidiones = ['Aprobado', 'Desistido','Renuncia de Licencia']

# Filtramos el DataFrame df_licencias para mantener solo las filas donde 'Tipo de decisión' esté en la lista decidiones.
#df_licencias = df_licencias.filter(col('Tipo de decisión').isin(decidiones))

# Filtramos el DataFrame df_licencias para mantener solo las filas donde 'Conteo licencias' sea mayor a 0.
df_licencias = df_licencias.filter(col('Conteo licencias') > 0)


In [0]:
# Agrupamos el DataFrame 'predios_agregados_df' por las columnas 'LOTLOTE_ID' y 'año_vigencia'.
# Luego, aplicamos varias funciones de agregación:
# - Sumamos los valores de la columna 'AREA_TERRENO'.
# - Sumamos los valores de la columna 'AREA_CONSTRUIDA'.
# - Sumamos los valores de la columna 'VALOR_AVALUO'.
# - Contamos el número de valores en la columna 'CHIP'.
# El resultado es un DataFrame que tiene una columna para cada función de agregación aplicada.
conteo_licencias = df_licencias.groupBy(["Código lote","Año de ejecutoría"]).agg({"Área":"sum", "Unidades":"sum", "Matrícula Inmobiliaria":"count", "CHIP":"count"}).fillna(0)
## Matrícula Inmobiliaria, CHIP, Unidades	Área
# Renombramos las columnas del DataFrame 'conteo_predios' para que tengan nombres más descriptivos y fáciles de entender.
# - "count(CHIP)" se renombra a "NumPredios".
# - "sum(VALOR_AVALUO)" se renombra a "ValorAvaluo".
# - "sum(AREA_CONSTRUIDA)" se renombra a "AC".
# - "sum(AREA_TERRENO)" se renombra a "AT".
conteo_licencias = conteo_licencias.withColumnRenamed("count(CHIP)","NumCHIP") \
    .withColumnRenamed("sum(Área)","Area") \
    .withColumnRenamed("sum(Unidades)","Unidades") \
    .withColumnRenamed("count(Matrícula Inmobiliaria)","Matriculas")


In [0]:
display(df_licencias)

In [0]:
# Agrupamos el DataFrame df_licencias por las columnas "Código lote" y "Año de ejecutoría".
# Luego, pivotamos el DataFrame usando la columna "ModalidadAgregada" para transformar sus valores únicos en columnas individuales.
# Posteriormente, contamos las ocurrencias de cada combinación de "Código lote", "Año de ejecutoría" y "ModalidadAgregada".
# Finalmente, reemplazamos cualquier valor nulo con 0 usando fillna(0).
conteo_modalidades = (df_licencias.groupBy(["Código lote","Año de ejecutoría"]).pivot("ModalidadAgregada").count()).fillna(0)


In [0]:
# Agrupamos el DataFrame df_licencias por las columnas "Código lote" y "Año de ejecutoría".
# Luego, pivotamos el DataFrame usando la columna "ModalidadAgregada" para transformar sus valores únicos en columnas individuales.
# Posteriormente, contamos las ocurrencias de cada combinación de "Código lote", "Año de ejecutoría" y "ModalidadAgregada".
# Finalmente, reemplazamos cualquier valor nulo con 0 usando fillna(0).
conteo_usos_licencias = (df_licencias.groupBy(["Código lote","Año de ejecutoría"]).pivot("UsoAgregado").count()).fillna(0)


In [0]:
display(conteo_usos_licencias)

In [0]:
# Convertimos el DataFrame de Spark (df_licencias) a un DataFrame de Pandas (df_licencias_pd).
df_licencias_pd = df_licencias.toPandas()

# Filtramos el DataFrame de Pandas para mantener solo las filas donde 'def_lat' no es nulo.
df_licencias_pd = df_licencias_pd[df_licencias_pd['def_lat'].notnull()]

# Filtramos el DataFrame de Pandas para mantener solo las filas donde 'def_lng' no es nulo.
df_licencias_pd = df_licencias_pd[df_licencias_pd['def_lng'].notnull()]

# Creamos una nueva columna 'geometry' en el DataFrame de Pandas.
# Esta columna contiene objetos Point (puntos geográficos) creados a partir de las columnas 'def_lng' y 'def_lat'.
df_licencias_pd['geometry'] = df_licencias_pd.apply(lambda row: Point(row['def_lng'], row['def_lat']), axis=1)

# Convertimos el DataFrame de Pandas (df_licencias_pd) a un GeoDataFrame (licencias_gdf) usando geopandas.
# Especificamos la columna 'geometry' como la columna que contiene las geometrías y definimos el sistema de referencia de coordenadas (CRS) como "EPSG:4326".
licencias_gdf = gpd.GeoDataFrame(df_licencias_pd, geometry='geometry', crs="EPSG:4326")


In [0]:
# Convertimos el DataFrame de Spark (isocrona_df) a un DataFrame de Pandas (isocronas_pd).
isocrona_df = isocrona_df.withColumn("geometry", col("geometry").cast("string"))
isocronas_pd = isocrona_df.toPandas()
isocronas_pd['geometry'] = isocronas_pd['geometry'].apply(wkt.loads)

In [0]:


# Convertimos el DataFrame de Pandas (isocronas_pd) a un GeoDataFrame (isocronas_gdf) usando geopandas.
# Especificamos la columna 'geometry' como la columna que contiene las geometrías y definimos el sistema de referencia de coordenadas (CRS) como "EPSG:4326".
isocronas_gdf = gpd.GeoDataFrame(isocronas_pd, geometry='geometry', crs="EPSG:4326")

# Realizamos una operación de join espacial entre los GeoDataFrames licencias_gdf e isocronas_gdf.
# El join se realiza con base en la relación espacial "within", es decir, se unen las filas de licencias_gdf que están dentro de las geometrías de isocronas_gdf.
# El resultado es un nuevo GeoDataFrame (licencias_gdf_iso) que contiene solo las licencias que están dentro de las isócronas.
licencias_gdf_iso = gpd.sjoin(licencias_gdf, isocronas_gdf, how="inner", op="within")

In [0]:
licencias_gdf_iso[['geometry','Tiempo']].sample(5000).explore('Tiempo', tiles='CartoDB positron')

In [0]:
campos_licencias = ['Código lote','Año de ejecutoría','Localidad','UPZ','Tiempo','layer','Linea','Tipo de decisión','def_lng','def_lat']
licencias_gdf = pd.DataFrame(licencias_gdf_iso)[campos_licencias].drop_duplicates()
conteo_licencias_df = conteo_licencias.toPandas().drop_duplicates()
conteo_modalidades_df = conteo_modalidades.toPandas().drop_duplicates()
conteo_usos_licencias_df = conteo_usos_licencias.toPandas().drop_duplicates()

In [0]:
# Realizamos una operación de unión (merge) entre el GeoDataFrame licencias_gdf_iso y el DataFrame conteo_modalidades.
# La unión se basa en las columnas 'Código lote' y 'Año de ejecutoría'.
# Utilizamos un join de tipo "left", lo que significa que se conservarán todas las filas del GeoDataFrame licencias_gdf_iso y se agregarán las columnas correspondientes del DataFrame conteo_modalidades.
# Si no hay coincidencia para una fila en particular de licencias_gdf_iso en conteo_modalidades, los valores de las columnas agregadas serán NaN.
union_licencias_df =  pd.merge(left=licencias_gdf, right=conteo_licencias_df, on=['Código lote','Año de ejecutoría'], how="left")
#union_licencias_df =  pd.merge(left=union_licencias_df, right=conteo_modalidades_df, on=['Código lote','Año de ejecutoría'], how="left")
union_licencias_df =  pd.merge(left=union_licencias_df, right=conteo_usos_licencias_df, on=['Código lote','Año de ejecutoría'], how="left")


In [0]:
union_licencias_df

In [0]:
#union_licencias_df.columns = ['CodigoLote','AnioDeEjecutoria','Localidad','UPZ','Tiempo','Estacion','Linea','TipoDeDecision','DefLng','DefLat','NumCHIP','Area','Unidades','Matriculas','Adecuacion','Ampliacion','Cerramiento','CulminacionDeObras','Demolicion','Modificacion','ObraNueva','Otras','PropiedadHorizontal','Reconocimiento','ReforzamientoEstructural','Restauración','SinModalidad','Subdivisión']
union_licencias_df.columns = ['CodigoLote','AnioDeEjecutoria','Localidad','UPZ','Tiempo','Estacion','Linea','TipoDeDecision','DefLng','DefLat','NumCHIP','Area','Unidades','Matriculas','ComercioYServicios','Industrial','Otros','Residencial']

In [0]:
# Convertir el DataFrame de Pandas a un DataFrame de PySpark
union_licencias_spark_df = spark.createDataFrame(union_licencias_df)

# Escribir el DataFrame de PySpark a un archivo CSV
union_licencias_spark_df.write.csv('/FileStore/tables/SDP/union_licencias_2008_2023.csv', header=True, mode="overwrite")

In [0]:
union_licencias_spark_df

In [0]:
union_licencias_spark_df.createOrReplaceTempView("union_sdp_spark_df")

In [0]:
%sql
CREATE OR REPLACE TABLE lotes_licencias_sdp AS (
  Select 
  *, cast(AnioDeEjecutoria as float) as AnioVigencia
  from union_sdp_spark_df where CodigoLote is not null);


## Sandbox